<img src = "https://drive.google.com/uc?export=view&id=1Hh_G3M13P9xSNgSiQ-WnALg93XwK_hG8" alt = "Encabezado MLDS" width = "100%">  </img>

# **Entrenamiento y Validación**
---

Este notebook es una plantilla que le puede servir como guía para el cuarto entregable del proyecto aplicado.


Proyecto aplicado por:


*   Cristhian David Mora Uribe cdmorau@unal.edu.co
*   Martin Camilo Rodriguez Murcia mrodriguezmu@unal.edu.co
*   Nestor Steven Negrete Pinilla narutones98@gmail.com

## **1. Entrenamiento del Modelo y Selección de Hiperparámetros**
---

Si está utilizando un modelo que requiere entrenamiento, deberá entrenarlo y seleccionar un conjunto de hiperparámetros válido para el mismo. Recuerde que tiene diversas erramientas para hacer validación.

In [ ]:
# VAMOS A CARGAR LA BASE DE DATOS PREPROCESADA

# conectamos con el drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# BASE DE DATOS
import pandas as pd

bitcoin_data = pd.read_csv('/content/drive/MyDrive/DeepLearning/bitcoin_preprocess.csv')


In [ ]:
bitcoin_data['Date'] = pd.to_datetime(bitcoin_data['Date'])

bitcoin_data = bitcoin_data.sort_values('Date')
# Convertimos a días desde una fecha base (1970-01-01)
bitcoin_data['Date'] = (bitcoin_data['Date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')

In [ ]:
bitcoin_data.head(5)

,Date,High,Low,Open,Close,Volume,Marketcap,Day_of_Week,Month,Year,High_Low_Diff,Close_Open_Diff,Close_Change_Percent,5_Day_MA,10_Day_MA,20_Day_MA,5_Day_Volatility,10_Day_Volatility
0,15843,0.000782,0.000914,0.000867,0.000868,0.0,0.000507,5,5,2013,0.000229,0.508551,0.468721,0.000720,0.000625,0.000510,0.001203,0.000438
1,15844,0.000771,0.000870,0.000862,0.000844,0.0,0.000493,6,5,2013,0.000383,0.508469,0.448584,0.000754,0.000640,0.000491,0.000869,0.000437
2,15845,0.000757,0.000879,0.000851,0.000844,0.0,0.000494,0,5,2013,0.000272,0.508517,0.463924,0.000779,0.000648,0.000477,0.000399,0.000470
3,15846,0.000748,0.000896,0.000843,0.000858,0.0,0.000502,1,5,2013,0.000139,0.508609,0.472822,0.000792,0.000660,0.000482,0.000122,0.000482
4,15847,0.000763,0.000909,0.000857,0.000874,0.0,0.000512,2,5,2013,0.000156,0.508618,0.474068,0.000795,0.000675,0.000497,0.000168,0.000479


In [ ]:
from sklearn.model_selection import train_test_split

# Definimos las características y la variable objetivo
X = bitcoin_data[['Open', 'High', 'Low', 'Volume']]
y = bitcoin_data['Close']  # Variable objetivo

# Dividimos el conjunto de datos en entrenamiento y prueba
train, test = train_test_split(bitcoin_data, test_size=0.20, shuffle=True)

In [ ]:
import numpy as np

# Definimos el tamaño de la ventana de tiempo
window_size = 60  # Número de días para mirar hacia atrás en cada predicción

from sklearn.preprocessing import MinMaxScaler

# Función para crear secuencias de datos
def secuencias(data, window_size):
    X = []
    y = []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:i+window_size][['Open', 'High', 'Low', 'Volume']].values)
        y.append(data.iloc[i+window_size]['Close'])
    return np.array(X), np.array(y)

In [ ]:
# Creamos secuencias de entrenamiento y prueba
X_train_sec, y_train_sec = secuencias(train, window_size)
X_test_sec, y_test_sec = secuencias(test, window_size)

# Cambiamos la forma de X para que sea compatible con Conv1D y LSTM

X_train_sec = X_train_sec.reshape((X_train_sec.shape[0], X_train_sec.shape[1], X_train_sec.shape[2], 1))
X_test_sec = X_test_sec.reshape((X_test_sec.shape[0], X_test_sec.shape[1], X_test_sec.shape[2], 1))

In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras_tuner import RandomSearch

# vamos a proceder al entrenamiento del modelo y a la selección de hiperparámetros. Para ello vamos a usar keras

# Definimos una función para construir el modelo CNN-LSTM con hiperparámetros variables
def build_model(hp):
    model = tf.keras.models.Sequential()

    # Capa Conv1D
    model.add(tf.keras.layers.Conv1D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size', values=[2, 3, 5]),
        activation='relu',
        padding='same',
        input_shape=(X_train_sec.shape[1], X_train_sec.shape[2])
    ))

    # Agregar más capas LSTM
    model.add(tf.keras.layers.LSTM(
        units=hp.Int('units', min_value=64, max_value=256, step=64),
        return_sequences=True  # Cambiar a True para agregar más capas LSTM
    ))

    model.add(tf.keras.layers.LSTM(
        units=hp.Int('units', min_value=64, max_value=256, step=64),
        return_sequences=False
    ))

    # Dropout
    model.add(tf.keras.layers.Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))

    # Capa densa para la salida
    model.add(tf.keras.layers.Dense(1))

    model.compile(optimizer=tf.keras.optimizers.Adam(
        learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='mse',
        metrics=['mae']
    )

    return model

In [ ]:
# Configuramos el Tuner de Keras para búsqueda aleatoria
tuner = RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=10,  # Número de combinaciones a probar
    executions_per_trial=2,  # Número de veces que se entrena cada configuración
    directory='hyperparam_tuning',
    project_name='crypto_price_prediction_cnn_lstm'
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Realizamos la búsqueda de hiperparámetros (# RUNNING TIME: 12 MIN CON GPU)
tuner.search(X_train_sec, y_train_sec, epochs=70, validation_data=(X_test_sec, y_test_sec), batch_size=32)

Trial 1 Complete [01h 50m 47s]
val_mae: 0.10196553170681

Best val_mae So Far: 0.10196553170681
Total elapsed time: 01h 50m 47s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
64                |128               |filters
2                 |2                 |kernel_size
128               |256               |units
0.2               |0.2               |dropout
0.001316          |0.0047123         |learning_rate

Epoch 1/70
73/73 ━━━━━━━━━━━━━━━━━━━━ 19s 212ms/step - loss: 0.0325 - mae: 0.1068 - val_loss: 0.0317 - val_mae: 0.1089
Epoch 2/70
73/73 ━━━━━━━━━━━━━━━━━━━━ 15s 208ms/step - loss: 0.0342 - mae: 0.1114 - val_loss: 0.0318 - val_mae: 0.1067
Epoch 3/70
73/73 ━━━━━━━━━━━━━━━━━━━━ 20s 204ms/step - loss: 0.0305 - mae: 0.1048 - val_loss: 0.0318 - val_mae: 0.1133
Epoch 4/70
73/73 ━━━━━━━━━━━━━━━━━━━━ 21s 206ms/step - loss: 0.0300 - mae: 0.1062 - val_loss: 0.0318 - val_mae: 0.1129
Epoch 5/70
73/73 ━━━━━━━━━━━━━━━━━━━━ 20s 204ms/step - loss: 0.0345 - mae: 0.

In [ ]:
# Obtenemos los mejores hiperparámetros
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Mejores hiperparámetros:")
for hp_name, hp_value in best_hps.values.items():
    print(f"{hp_name}: {hp_value}")


In [ ]:
# Obtenemos el mejor modelo
best_model = tuner.hypermodel.build(best_hps)

In [ ]:
!pip install tensorflow.keras.callbacks

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = best_model.fit(X_train_sec, y_train_sec,
                          epochs=500,
                          validation_data=(X_test_sec, y_test_sec),
                          batch_size=32,
                          callbacks=[early_stopping])

## **2. Evaluación o Aplicación del modelo**
---

Si entrenó un modelo, recuerde que debe reportar el desempeño del mismo sobre un conjunto de datos no visto (test). Considere que dispone de las siguientes métricas:

- **Clasificación**: accuracy, precision, recall, f1-score, AUC.
- **Regresión**: $r^2$, error cuadrático medio, error absoluto medio.
- **Agrupamiento**: coeficiente de silueta, índice de Davies-Bouldin.

In [ ]:
# ---**INGRESE SU CÓDIGO**---

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Realizamos predicciones sobre el conjunto de prueba
y_pred = best_model.predict(X_test_sec)

# Calculamos las métricas de evaluación
mse = mean_squared_error(y_test_sec, y_pred)
mae = mean_absolute_error(y_test_sec, y_pred)
r2 = r2_score(y_test_sec, y_pred)

# Imprimimos los resultados
print(f"Error Cuadrático Medio (MSE): {mse}")
print(f"Error Absoluto Medio (MAE): {mae}")
print(f"Coeficiente de Determinación (R^2): {r2}")

Adicionalmente, si utilizó un modelo pre-entrenado, debe generar predicciones y mostrar el resultado final de la aplicación del modelo sobre sus datos:

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/DeepLearning/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# Descargar el dataset de "Twitter Financial News" desde Kaggle y extraerlo
!kaggle datasets download sudalairajkumar/cryptocurrencypricehistory
import os
import zipfile
import pandas as pd

# Extraer el archivo zip descargado
with zipfile.ZipFile("/content/cryptocurrencypricehistory.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/crypto_data")

# Ruta donde están los archivos extraídos
folder_path = "/content/crypto_data/"

# Lista de los archivos CSV que quieres leer
file_names = [
    "coin_Aave.csv", "coin_BinanceCoin.csv", "coin_Bitcoin.csv", "coin_Cardano.csv",
    "coin_ChainLink.csv", "coin_Cosmos.csv", "coin_CryptocomCoin.csv", "coin_Dogecoin.csv",
    "coin_EOS.csv", "coin_Ethereum.csv", "coin_Iota.csv", "coin_Litecoin.csv",
    "coin_Monero.csv", "coin_NEM.csv", "coin_Polkadot.csv", "coin_Solana.csv",
    "coin_Stellar.csv", "coin_Tether.csv", "coin_Tron.csv", "coin_USDCoin.csv",
    "coin_Uniswap.csv", "coin_WrappedBitcoin.csv", "coin_XRP.csv"
]

# Diccionario para almacenar los DataFrames
dataframes = {}

# Leer cada archivo CSV y almacenarlo en el diccionario
for file in file_names:
    file_path = os.path.join(folder_path, file)
    coin_name = file.replace("coin_", "").replace(".csv", "")  # Extraer el nombre de la criptomoneda
    dataframes[coin_name] = pd.read_csv(file_path)

scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(dataframes['Bitcoin'][['Close']].values)

In [ ]:
y_test_sec1 = y_test_sec.reshape(-1, 1)

In [ ]:
y_test_sec1 = scaler.inverse_transform(y_test_sec1)

In [ ]:
y_pred1 = scaler.inverse_transform(y_pred)

In [ ]:
# ---**INGRESE SU CÓDIGO**---


# Convertimos las predicciones a un DataFrame
results = pd.DataFrame({
    'Fecha': bitcoin_data['Date'].iloc[len(train) + window_size:].values,
    'Valor Real': y_test_sec1.flatten(),
    'Predicción': y_pred1.flatten()  # Aplanamos para que sea una lista unidimensional
})

# Mostramos los primeros 10 resultados
print(results.head(10))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(results['Fecha'], results['Valor Real'], label='Valor Real', color='blue')
plt.plot(results['Fecha'], results['Predicción'], label='Predicción', color='orange')
plt.title('Predicción del Precio de Bitcoin')
plt.xlabel('Fecha')
plt.ylabel('Precio de Cierre')
plt.legend()
plt.show()

# **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistentes docentes :**
  * [Santiago Toledo Cortés](https://sites.google.com/unal.edu.co/santiagotoledo-cortes/)
* **Diseño de imágenes:**
    - [Mario Andres Rodriguez Triana](mailto:mrodrigueztr@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*